In [0]:
%sql
describe globalretail_bronze.bronze_customers;

In [0]:
spark.sql("USE globalretail_silver")
spark.sql("""
    CREATE TABLE IF NOT EXISTS silver_customers (
    customer_id STRING,
    name STRING,
    email STRING,
    country STRING,
    customer_type STRING,
    registration_date DATE,
    age INT,
    gender STRING,
    total_purchases INT,
    customer_segment STRING,
    days_since_registration INT,
    last_updated TIMESTAMP)
""")

In [0]:
%sql
select table_name 
from workspace.information_schema.tables 
where table_schema = 'globalretail_silver'

In [0]:
%sql
describe globalretail_silver.silver_customers;